In [ ]:
import os
import cv2

# Diretórios
LABEL_DIR = r"C:\Users\jonas\Downloads\dota_dataset\test\labelTxt"
IMAGE_DIR = r"C:\Users\jonas\Downloads\dota_dataset\test\images"
OUTPUT_DIR = r"C:\Users\jonas\Downloads\dota_dataset\test\labels_hbb"

# Classes do DOTA (pode ajustar se quiser filtrar)
CLASSES = [
    'plane', 'ship', 'storage-tank', 'baseball-diamond', 'tennis-court',
    'basketball-court', 'ground-track-field', 'harbor', 'bridge', 'large-vehicle',
    'small-vehicle', 'helicopter', 'roundabout', 'soccer-ball-field', 'swimming-pool',
    'container-crane'
]

# Garante que a pasta de saída existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

def obb_to_hbb(points):
    """Converte uma lista de 8 números (4 pontos) para HBB (xmin, ymin, xmax, ymax)"""
    x_coords = points[0::2]
    y_coords = points[1::2]
    return min(x_coords), min(y_coords), max(x_coords), max(y_coords)

def convert_label_file(label_path, image_path, output_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 10 or parts[8] == '###':
            continue  # ignora linhas inválidas ou objetos marcados como "difficult"

        points = list(map(float, parts[:8]))
        class_name = parts[8]
        if class_name not in CLASSES:
            continue

        class_id = CLASSES.index(class_name)
        xmin, ymin, xmax, ymax = obb_to_hbb(points)

        # converte para formato YOLO (x_center, y_center, w, h) normalizado
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        bbox_width = (xmax - xmin) / width
        bbox_height = (ymax - ymin) / height

        yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
        new_lines.append(yolo_line)

    # Salva no novo formato
    with open(output_path, 'w') as f_out:
        for l in new_lines:
            f_out.write(l + '\n')

# Processa todos os arquivos
label_files = [f for f in os.listdir(LABEL_DIR) if f.endswith('.txt')]
print(f"Convertendo {len(label_files)} arquivos de rótulo...")

for label_file in label_files:
    image_file = label_file.replace('.txt', '.png')
    label_path = os.path.join(LABEL_DIR, label_file)
    image_path = os.path.join(IMAGE_DIR, image_file)
    output_path = os.path.join(OUTPUT_DIR, label_file)

    if not os.path.exists(image_path):
        print(f"Imagem não encontrada: {image_path}")
        continue

    convert_label_file(label_path, image_path, output_path)

print("Conversão concluída. Arquivos salvos em:")
print(OUTPUT_DIR)